
# Trabajo Practico 3 ***Procesamiento***



### **Proceso** el data frame de Markets


In [3]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta

In [ ]:
df_markets = pd.read_parquet("datalake_Santi/raw_date/coincap/markets/markets.parquet")
df_markets = df_markets.dropna(subset=['tradesCount24Hr']) # Elimino las filas cuyo valor de "tradesCount24Hr" no representa valor. Son la cantidad de operaciones en las ultimas 24 horas
df_markets = df_markets.dropna(subset=['priceUsd'])
df_markets = df_markets.dropna(subset=['percentExchangeVolume'])
df_markets = df_markets.dropna(subset=['baseId'])
df_markets = df_markets.dropna(subset=['exchangeId'])
df_market = df_markets.drop_duplicates()
df_markets["priceUsd"] = pd.to_numeric(df_markets["priceUsd"])
df_markets["updated"] = pd.to_datetime(df_markets["updated"]) #Formateo la fecha a %d-%m-%Y %H:%M
df_markets["updated"] = df_markets["updated"].dt.strftime('%d-%m-%Y %H:%M') #Formateo la fecha a %d-%m-%Y %H:%M
df_markets['priceUsd'] = df_markets['priceUsd'].astype(float)
df_markets['priceUsd'] = df_markets['priceUsd'].round(decimals=4)

df_markets_g = df_markets.groupby(
     ["exchangeId","baseId"],
    as_index=False).agg(
        {
            "priceUsd": ["mean", "max", "min"]
        }
)
df_markets_g["priceUsd","mean"] = df_markets_g["priceUsd","mean"].round(decimals=4)
df_markets_g

### **Proceso** el data frame de Exchanges


In [133]:
df_exchanges = pd.read_parquet("datalake_Santi/raw_date/coincap/exchanges")
df_exchanges = df_exchanges.dropna(subset=['percentTotalVolume']) # Elimino las filas cuyo valor de "Porcentaje Total del volumen" no representa valor.
df_exchanges = df_exchanges.drop_duplicates() #Eliminos las filas duplicadas
#Redondeo los valores float a los minimos decimales necesarios
df_exchanges['percentTotalVolume'] = df_exchanges['percentTotalVolume'].astype(float)
df_exchanges['volumeUsd'] = df_exchanges['volumeUsd'].astype(float)
df_exchanges['percentTotalVolume'] = df_exchanges['percentTotalVolume'].round(decimals=7)
df_exchanges['volumeUsd'] = df_exchanges['volumeUsd'].round(decimals=5)
df_exchanges["updated"] = pd.to_datetime(df_exchanges["updated"]) #Formateo la fecha a %d-%m-%Y %H:%M
df_exchanges["updated"] = df_exchanges["updated"].dt.strftime('%d-%m-%Y %H:%M') #Formateo la fecha a %d-%m-%Y %H:%M
df_exchanges["fecha"] = pd.to_datetime(df_exchanges["fecha"]) #Formateo la fecha a %d-%m-%Y %H:%M
df_exchanges["fecha"] = df_exchanges["fecha"].dt.strftime('%d-%m-%Y') #Formateo la fecha a %d-%m-%Y %H:%M
# df_exchanges.head(200)

### **Proceso** el data frame de Assets


In [ ]:
df_assets = pd.read_parquet("datalake_Santi/raw_date/coincap/assets")
df_assets = df_assets.dropna(subset=['maxSupply']) # Elimino las filas cuyo valor de "Porcentaje Total del volumen" no representa valor.
df_assets = df_assets.drop_duplicates() #Eliminos las filas duplicadas
#Redondeo los valores float a los minimos decimales necesarios
df_assets['supply'] = df_assets['supply'].astype(float)
df_assets['supply'] = df_assets['supply'].round(decimals=2)
df_assets['maxSupply'] = df_assets['maxSupply'].astype(float)
df_assets['maxSupply'] = df_assets['maxSupply'].round(decimals=2)
df_assets['marketCapUsd'] = df_assets['marketCapUsd'].astype(float)
df_assets['marketCapUsd'] = df_assets['marketCapUsd'].round(decimals=2)
df_assets['volumeUsd24Hr'] = df_assets['volumeUsd24Hr'].astype(float)
df_assets['volumeUsd24Hr'] = df_assets['volumeUsd24Hr'].round(decimals=2)
df_assets['priceUsd'] = df_assets['priceUsd'].astype(float)
df_assets['priceUsd'] = df_assets['priceUsd'].round(decimals=2)
df_assets['changePercent24Hr'] = df_assets['changePercent24Hr'].astype(float)
df_assets['changePercent24Hr'] = df_assets['changePercent24Hr'].round(decimals=2)
df_assets['vwap24Hr'] = df_assets['vwap24Hr'].astype(float)
df_assets['vwap24Hr'] = df_assets['vwap24Hr'].round(decimals=2)
df_assets.rename(columns={'id': 'baseId'}, inplace=True)
join_assets_markets = pd.merge(df_assets, df_markets, on='baseId', how='inner') #Hago un JOIN entre markets y assets para saber el suministro disponible para el comercio y el volumen negociado en dolares de las ultimas 24 horas
join_assets_markets

### **Proceso** el data frame de Concentrations


In [ ]:
df_Concentrations = pd.read_parquet("datalake_Santi/raw_date/luchtmeetnet/concentrations/N02")
df_Concentrations = df_Concentrations.drop_duplicates() #Eliminos las filas duplicadas
df_Concentrations["timestamp_measured"] = pd.to_datetime(df_Concentrations["timestamp_measured"])
df_Concentrations["timestamp_measured"] = df_Concentrations["timestamp_measured"].dt.strftime('%d-%m-%Y %H:%M') #Formateo la fecha a %d-%m-%Y %H:%M
df_Concentrations["fecha"] = pd.to_datetime(df_Concentrations["fecha"])
df_Concentrations["fecha"] = df_Concentrations["fecha"].dt.strftime('%Y-%m-%d') #Formateo la fecha a %d-%m-%Y
df_Concentrations['value_is_greater_than_20'] = df_Concentrations['value']>20 #creo una columna que nos indica si el valor de value supera el valor 20
df_Concentrations = df_Concentrations.drop(columns=['timestamp_measured'])
df_Concentrations

## ***Almacenamiento en la Base de datos OLAP asignada***

In [ ]:
!pip install psycopg[binary]
!pip install sqlalchemy==1.4.49

In [140]:

import sqlalchemy as sa
import os
from configparser import ConfigParser

def connect_to_postgres(config_file_path="config.ini", section="postgres"):
    """
    Establece una conexión a una base de datos postgres
    utilizando la configuración especificada en un archivo INI.

    Args:
        config_file_path (str, optional): Ruta del archivo de configuración INI. Por defecto, "config.ini".
        section (str, optional): Nombre de la sección en el archivo INI que contiene los datos de conexión. Por defecto, "postgres".

    Returns:
        sqlalchemy.engine.Engine: La conexión a la base de datos de Redshift.
    """
    # Comprobar si el archivo de configuración existe
    if not os.path.exists(config_file_path):
        raise FileNotFoundError(f"El archivo de configuración '{config_file_path}' no existe.")

    # Leer la configuración desde el archivo INI
    config = ConfigParser()
    config.read(config_file_path)
    conn_data = config[section]

    # Obtener los parámetros de conexión
    host = conn_data.get("host")
    port = conn_data.get("port")
    db = conn_data.get("db")
    user = conn_data.get("user")
    pwd = conn_data.get("pwd")

    url = f"postgresql://{user}:{pwd}@{host}:{port}/{db}"

    # Establecer la conexión a la base de datos PostgreSQL
    conn = sa.create_engine(url)

    return conn


engine = connect_to_postgres("config.ini") # Me conecto a la base de datos donde voy a realizar la creacion de la tabla

### **Creo la tabla para el Data Frame procesado de Concentrations** e introduzco los datos en la tabla


In [124]:
# formula	value	timestamp_measured	fecha	hora	value_is_greater_than_20
engine.execute("""
               CREATE TABLE IF NOT EXISTS santiagoarab_luchtmeetnet_concentrations(
                formula VARCHAR(10),
                value FLOAT,
                fecha DATE,
                hora INT,
                value_is_greater_than_20 BOOLEAN,
                PRIMARY KEY(formula,fecha,hora)
               )
               """)  # Creo la tabla en la base de datos

C:\Users\Santi\AppData\Local\Temp\ipykernel_18332\3467055731.py:2: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  engine.execute("""


In [ ]:
engine.execute("""
                TRUNCATE TABLE santiagoarab_luchtmeetnet_concentrations

               """)
df_Concentrations.to_sql("santiagoarab_luchtmeetnet_concentrations", engine,
                         method = "multi", if_exists = "append", index=False)
engine.execute("""
                SELECT * FROM santiagoarab_luchtmeetnet_concentrations
               """).fetchall()

### Guardo los datos extraido de las tablas restantes usando el formato por defecto que proporciona pandas

In [ ]:
engine.execute("""
                TRUNCATE TABLE santiagoarab_coincap_assets

               """)
join_assets_markets.to_sql("santiagoarab_coincap_assets", engine,method = "multi", if_exists = "append")
engine.execute("""
                SELECT * FROM santiagoarab_coincap_assets
               """).fetchall()

In [ ]:
engine.execute("""
                TRUNCATE TABLE santiagoarab_coincap_exchanges

               """)
df_exchanges.to_sql("santiagoarab_coincap_exchanges", engine,method = "multi", if_exists='append')
engine.execute("""
                SELECT * FROM santiagoarab_coincap_exchanges
               """).fetchall()

In [ ]:
engine.execute("""
                TRUNCATE TABLE santiagoarab_coincap_markets

               """)

df_markets_g.to_sql("santiagoarab_coincap_markets", engine ,method = "multi", if_exists = "append" , index=False)
engine.execute("""
                SELECT * FROM santiagoarab_coincap_markets
               """).fetchall()